# Research Software Engineering
--------------------------------
## Real Estate Price Analysis

### Why this topic?
House price prediction is of significant importance in today's world. It plays a crucial role in financial planning for individuals and families, helping them make informed decisions about buying or selling properties. For investors, it uncovers potential profitable opportunities. Financial institutions rely on it for risk management in mortgage lending. Economists and policymakers use it as a key tool for economic analysis and policy development. With advancements in AI and machine learning, the accuracy of these predictions has improved, making it an exciting and vital field in the current data-driven era.

### About the dataset
This dataset, obtained from [Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques) for the house pricing challenge, is the focus of this project. The objective is to delve into the training dataset through preprocessing steps, which encompass data mining methods such as discarding unnecessary data and filling in missing values. Following this, a range of machine learning algorithms will be applied to the refined dataset. The end goal is to identify the most effective model for forecasting property prices, taking into account various characteristics, including aspects like location.

### Import Libraries

In [50]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore")

### Load Dataset

We are using pandas to read the dataset from local directory. It is easy to use and analyze data.

In [51]:
data = pd.read_csv("..\\data\\real_estate_data.csv")
data.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


We can see that first five row of the dataset. We see that some columns contain NaN and 0 which we need to handle. But first let's see the shape and the columns values and their type in dataset.

In [52]:
print("The shape of our dataset is: ", data.shape)
data.info()

The shape of our dataset is:  (1460, 81)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual

Some of the columns contains missing values and some columns we dont need at all such as column "Id". Before we proceed with cleaning the columns with the most missing data, we will using a map to change some categorical data to numrical data.
We checked manually the description.txt, and we conclude we can use a mapping for some categorical data to change them to numerical data

In [53]:
data = data.drop('Id', axis=1)
mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
columns_to_map = ['GarageQual', 'GarageCond', 'PoolQC', 'FireplaceQu', 'KitchenQual', 'HeatingQC', 'BsmtCond', 'BsmtQual', 'ExterCond', 'ExterQual']
for column in columns_to_map:
    data[column] = data[column].map(mapping)

We notice that we have two column, Year built and Year sold, we can drop these two column and create a new one for Age
that means how old is the house.

In [54]:
columns = data.columns.tolist()
columns.insert(-1, 'Age')
data['Age'] = data['YrSold'] - data['YearBuilt']
columns.remove('YearBuilt')
columns.remove('YrSold')
data = data[columns]

data.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,SaleType,SaleCondition,Age,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,WD,Normal,5,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,WD,Normal,31,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,WD,Normal,7,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,WD,Abnorml,91,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,WD,Normal,8,250000


Now we convert the NaN values to zero in order to calculate efficiently.

In [55]:
data = data.fillna(0)
data.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,SaleType,SaleCondition,Age,SalePrice
0,60,RL,65.0,8450,Pave,0,Reg,Lvl,AllPub,Inside,...,0,0.0,0,0,0,2,WD,Normal,5,208500
1,20,RL,80.0,9600,Pave,0,Reg,Lvl,AllPub,FR2,...,0,0.0,0,0,0,5,WD,Normal,31,181500
2,60,RL,68.0,11250,Pave,0,IR1,Lvl,AllPub,Inside,...,0,0.0,0,0,0,9,WD,Normal,7,223500
3,70,RL,60.0,9550,Pave,0,IR1,Lvl,AllPub,Corner,...,0,0.0,0,0,0,2,WD,Abnorml,91,140000
4,60,RL,84.0,14260,Pave,0,IR1,Lvl,AllPub,FR2,...,0,0.0,0,0,0,12,WD,Normal,8,250000


Also, by observing the dataset (after running the shape function), there are colums with many missing data as such alley (only 91/1461 non-null).
So we will count the missing data in each colums.

In [56]:
def count_null_data(data):
    missing_counts = (data == 0).sum()
    sorted_columns = missing_counts.sort_values(ascending=False)
    no_missing_data = True
    for column, count in sorted_columns.items():
        if pd.api.types.is_numeric_dtype(data[column]):
            nan_count = data[column].isna().sum()
            count += nan_count
        if count != 0:
            print(f"Column '{column}': {count} values 0")
            no_missing_data = False
    if no_missing_data:
        print("There are no 0 value anymore!")


count_null_data(data)

Column 'PoolArea': 1453 values 0
Column 'PoolQC': 1453 values 0
Column '3SsnPorch': 1436 values 0
Column 'LowQualFinSF': 1434 values 0
Column 'MiscVal': 1408 values 0
Column 'MiscFeature': 1406 values 0
Column 'BsmtHalfBath': 1378 values 0
Column 'Alley': 1369 values 0
Column 'ScreenPorch': 1344 values 0
Column 'BsmtFinSF2': 1293 values 0
Column 'EnclosedPorch': 1252 values 0
Column 'Fence': 1179 values 0
Column 'HalfBath': 913 values 0
Column 'MasVnrType': 872 values 0
Column 'MasVnrArea': 869 values 0
Column 'BsmtFullBath': 856 values 0
Column '2ndFlrSF': 829 values 0
Column 'WoodDeckSF': 761 values 0
Column 'FireplaceQu': 690 values 0
Column 'Fireplaces': 690 values 0
Column 'OpenPorchSF': 656 values 0
Column 'BsmtFinSF1': 467 values 0
Column 'LotFrontage': 259 values 0
Column 'BsmtUnfSF': 118 values 0
Column 'GarageArea': 81 values 0
Column 'GarageType': 81 values 0
Column 'GarageCars': 81 values 0
Column 'GarageQual': 81 values 0
Column 'GarageCond': 81 values 0
Column 'GarageYrBl

Depending of the number of missing data in each columns of our numerical data, we will use a threshold to remove the column with the most missing data. For example, most of the column has more than 400 input data points so we will set the 
threshold value of 400. The columns contain more that 400 zero values will be dropped from the dataset. Because the zero values will not increase the performance of the model rather decrease the it.

In [57]:
def delete_columns_with_zero_data(data, threshold):
    for column in data.columns:
        zero_count = (data[column] == 0).sum()
        if zero_count > threshold:
            data = data.drop(column, axis=1)
    return data

threshold = 400
data = delete_columns_with_zero_data(data, threshold)
count_null_data(data)

Column 'LotFrontage': 259 values 0
Column 'BsmtUnfSF': 118 values 0
Column 'GarageType': 81 values 0
Column 'GarageFinish': 81 values 0
Column 'GarageCars': 81 values 0
Column 'GarageArea': 81 values 0
Column 'GarageQual': 81 values 0
Column 'GarageCond': 81 values 0
Column 'GarageYrBlt': 81 values 0
Column 'Age': 64 values 0
Column 'BsmtExposure': 38 values 0
Column 'BsmtFinType2': 38 values 0
Column 'BsmtQual': 37 values 0
Column 'BsmtCond': 37 values 0
Column 'TotalBsmtSF': 37 values 0
Column 'BsmtFinType1': 37 values 0
Column 'FullBath': 9 values 0
Column 'BedroomAbvGr': 6 values 0
Column 'Electrical': 1 values 0
Column 'KitchenAbvGr': 1 values 0


As we can see, most of the columns has been dropped from the dataset as they contained more than 400 zero values. Now Before proceeding with the further, we will split out dataset to numerical and categorical data
As we checked also, that the remaining categorical data (part of them has been converted to numerical data) does not have any impact on our dataset

In [58]:
def separate_categorical_numerical(data):
    categorical_cols = []
    numerical_cols = []
    for column in data.columns:
        if data[column].dtype == 'object' or pd.api.types.is_categorical_dtype(data[column].dtype):
            categorical_cols.append(column)
        else:
            numerical_cols.append(column)
    return categorical_cols, numerical_cols


categorical_cols, numerical_cols = separate_categorical_numerical(data)
numerical_data = data[numerical_cols].copy()
categorical_data = data[categorical_cols].copy()

Let's see the both categorical and numerical columns

In [59]:
print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

Categorical columns: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'Foundation', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageType', 'GarageFinish', 'PavedDrive', 'SaleType', 'SaleCondition']
Numerical columns: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'MoSold', 'Age', 'SalePrice']
